# **DE - Business Intelligence - ETL**

In [1]:
!pip install gspread==4.0.1

In [2]:
pip install --upgrade oauth2client

Note: you may need to restart the kernel to use updated packages.


## **Setup Library & Import Library**

In [3]:
#library dataframe
import pandas as pd 
import numpy as np
import datetime as dt

#library gdrive
import json
import gspread
from oauth2client.service_account import ServiceAccountCredentials

#ignore library
import warnings
warnings.filterwarnings('ignore')

## **Setup Google Connection**

In [4]:
with open('digitalskola_new.json','rb') as file:
    key = json.load(file)
    
scope = ['https://www.googleapis.com/auth/drive','https://spreadsheets.google.com/feeds']
creds = ServiceAccountCredentials.from_json_keyfile_dict(key, scope)
client = gspread.authorize(creds)

## **Connection Source Data**

In [5]:
sheet = client.open('DE - Business Intelligence - Example Data')

## **ETL Process**

#### **>>> Extract Process**

In [6]:
# Memastikan Proses saja tidak dipakai
# masterSheet = pd.read_excel('tb_master_sheet.xlsx')
# masterSheet.head(1)

In [7]:
#Memastikan Proses saja tidak dipakai
# masterSheet = pd.read_excel('tb_master_sheet.xlsx')
# for index,raw in masterSheet.iterrows():
#    print(raw['Use'])

In [8]:
masterSheet = pd.read_excel('tb_master_sheet.xlsx')

data = pd.DataFrame([])
for index,raw in masterSheet.iterrows():
    if raw['Status'] != 0:
        tab = sheet.worksheet(raw['Use'])
        df = pd.DataFrame(tab.get_all_records())
        print(f"Raw Data sheet {raw['Use']} : {df.shape[0]} ")
        data = data.append(df)
data = data.reset_index()
print(f"Raw Data All : {data.shape[0]}")

Raw Data sheet JANUARI : 1940 
Raw Data sheet FEBRUARI : 1556 
Raw Data sheet MARET : 1799 
Raw Data sheet APRIL : 2072 
Raw Data sheet MEI : 2167 
Raw Data sheet JUNI : 1834 
Raw Data sheet JULI : 1854 
Raw Data sheet AGUSTUS : 1463 
Raw Data sheet SEPTEMBER : 204 
Raw Data All : 14889


#### **>>> Explorasi data analisis**

In [9]:
da = data.copy()

In [10]:
da['KODE DONATUR'] = da['KODE DONATUR'].replace({'':None})

In [11]:
da[da['TANGGAL'] == '1 Jan 2021'].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55 entries, 0 to 1897
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   index         55 non-null     int64 
 1   KODE DONATUR  20 non-null     object
 2   TANGGAL       55 non-null     object
 3   NO REFF       55 non-null     object
 4    DEBET        55 non-null     object
 5   KETERANGAN    55 non-null     object
 6   NAMA BANK     55 non-null     object
dtypes: int64(1), object(6)
memory usage: 3.4+ KB


In [12]:
da[da['TANGGAL'] == '1 Jan 2021']['KODE DONATUR'].unique()

array([441, 725, 32, None, 265, 1623, 1143, 966, 858, 1082, 1096, 1087,
       1101, 1084, 622, 806, 754, 348, 1019], dtype=object)

#### **>>> Transform Process**

In [13]:
def debetFunc(x):
    return int(x.replace('Rp','').replace('.','').strip())

In [14]:
#replace null value from source data
data['KODE DONATUR'] = data['KODE DONATUR'].replace({'':None})
data[' DEBET'] = data[' DEBET'].replace({'':0})

#format value int
for index,raw in data.iterrows():
    try:
        data.loc[index, 'DEBET'] = debetFunc(raw[' DEBET'])
    except:
        pass
#agg
result = data.groupby(['TANGGAL']) \
            .agg({'KODE DONATUR':'nunique', 
                  'NO REFF':'count',
                  'DEBET':'sum'}) \
            .reset_index()
 
result = result.rename(columns = {'KODE DONATUR':'Total Donatur',
                                  'NO REFF':'Total Transaksi',
                                  'DEBET':'Total Amount'})

#### **>>> Load Process**

In [15]:
export = sheet.worksheet('Aggregasi')
export.update([result.columns.values.tolist()] + result.values.tolist())

{'spreadsheetId': '1cQ9pHVY_Jv0-e9yoe5ST3hIJuQMKbmngfnQg4-El2Vc',
 'updatedRange': 'Aggregasi!A1:D260',
 'updatedRows': 260,
 'updatedColumns': 4,
 'updatedCells': 1040}